## Ejercicio 7

Utilizando los atributos numéricos del archivo Autos.csv entrene un combinador lineal para predecir el
precio del auto en función del resto de los atributos. Tenga en cuenta la existencia de valores faltantes.
Realice varias ejecuciones y observe los vectores de pesos obtenidos. ¿Puede identificar cuáles son los
atributos con mayor incidencia en la determinación del precio? Explique.